In [1]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

c:\Users\povhi\anaconda3\envs\dsa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [4]:
docs = fetch_20newsgroups(subset='train',  remove=('headers', 'footers', 'quotes'))

In [19]:
import json

with open("example_data.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

result_dict = {k: [item[k] for item in data_list] for k in data_list[0].keys()}

result_dict['full_text'] = [item['title'] + ' ' + item['content'] for item in data_list]

print(result_dict)

{'date': ['15.01.2025', '01.02.2025', '31.01.2025', '16.12.2022'], 'type': ['blog', 'blog', 'press_release', 'press_release'], 'authors': [['Matteo Ciccarelli', 'Matthieu Darracq Pariès', 'Bettina Landau', 'João Sousa'], ['Christine Lagarde', 'Ursula von der Leyen'], ['ECB'], ['ECB']], 'title': ['Exploring an uncertain future with the help of scenarios', 'Europe has strengths it can build on', 'ECB and ESRB issue joint report on experiences of using the countercyclical capital buffer early in the cycle', 'ECB publishes enhanced rules for private financial transactions of high-level officials'], 'content': ["Central banks project future developments based on past data patterns and a set of assumptions. Crises can change economic structures, complicating this forecasting. The ECB Blog explains how scenario, risk and sensitivity analyses address the new uncertainty. We use economic models and data patterns from the past to project the future. In normal times, assessing what conditions wil

In [ ]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(result_dict['content'])

In [8]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3736,-1_the_to_of_and,"[the, to, of, and, is, in, it, that, you, for]","[\nTo answer your irrelevant question, yes a p..."
1,0,1068,0_team_game_he_season,"[team, game, he, season, games, players, play,...","[\n\tOh, yeah. Dave Winfield--marginal player..."
2,1,512,1_patients_msg_of_medical,"[patients, msg, of, medical, is, in, it, healt...",[------------- cut here -----------------\n\n\...
3,2,371,2_key_chip_clipper_encryption,"[key, chip, clipper, encryption, keys, escrow,...",[I have an idea as to why the encryption algor...
4,3,311,3_hello_ites_hi_ditto,"[hello, ites, hi, ditto, huh, each, why, of, , ]","[ites:, Hello,, Hello,]"
...,...,...,...,...,...
137,136,11,136_plastic_paint_wax_scratches,"[plastic, paint, wax, scratches, car, polish, ...",[\nSuggest McQuires #1 plastic polish. It wil...
138,137,11,137_modem_com_16550_duo,"[modem, com, 16550, duo, windows, uart, turboc...",[I have had a Gateway Telepath modem for about...
139,138,10,138_celp_voice_dsp_speech,"[celp, voice, dsp, speech, 4800, coder, bps, s...",[\nA good vocoder like ours will give you 8000...
140,139,10,139_groups_windows_desktop_manager,"[groups, windows, desktop, manager, winfileini...","[Sorry, I did`nt tell exactly what I need.\n\n..."


## Scraping press release data

In [ ]:
# donwload the html files/pdfs of th ecb speeches
import pandas as pd

df = pd.read_csv(r"data\export_datamart.csv")
df['when_speech'] = pd.to_datetime(df['when_speech'], format="%Y-%m-%d")
display(df['what_type'].value_counts())

for i in df['what_type'].unique(): 
    print(i, df[df['what_type'] == i]['what_weblink'].iloc[-1])

df['file_ending'] = df['what_weblink'].str.extract(r'\.(\w+)$')
display(df['file_ending'].value_counts())
display(df.head())

import matplotlib.pyplot as plt

df.groupby([df['when_speech'].dt.year, 'what_type']) \
  .size() \
  .unstack(fill_value=0) \
  .plot(kind='bar', stacked=True)
plt.title("Counts by Year (stacked by what_type)")
plt.show()

what_type
S    2883
I     565
P     284
B     150
E     101
Name: count, dtype: int64

S https://www.ecb.europa.eu/press/key/date/2025/html/ecb.sp250210~77679a7d66.en.html
P https://www.ecb.europa.eu/press/press_conference/monetary-policy-statement/2025/html/ecb.is250130~1f418aa0f4.en.html
I https://www.ecb.europa.eu/press/inter/date/2025/html/ecb.in250206~69a68c13da.en.html
E https://www.ecb.europa.eu/press/tvservices/podcast/html/ecb.pod250130_episode101.en.html
B https://www.ecb.europa.eu/press/blog/date/2025/html/ecb.blog20250205~44578cf53f.en.html


file_ending
html    3870
pdf       25
Name: count, dtype: int64

,speech_id,when_speech,who,what_title,what_frequencies,what_language,what_weblink,what_type,file_ending
0,1,1997-02-07,Alexandre Lamfalussy,Conference organised by the Hungarian Banking ...,"{""banking"":31,""financial"":21,""bank"":17,""inflat...",EN,https://www.ecb.europa.eu/press/key/date/1997/...,S,html
1,2,1997-03-10,Alexandre Lamfalussy,Securing the benefits of EMU,"{""emu"":42,""monetary"":31,""policy"":19,""benefits""...",EN,https://www.ecb.europa.eu/press/key/date/1997/...,S,html
2,3,1997-04-22,Alexandre Lamfalussy,Convergence and the role of the European Centr...,"{""euro"":33,""escb"":21,""policy"":20,""monetary"":19...",EN,https://www.ecb.europa.eu/press/key/date/1997/...,S,html
3,4,1997-04-30,Alexandre Lamfalussy,The operation of monetary policy in stage thre...,"{""monetary"":54,""escb"":54,""policy"":45,""strategy...",EN,https://www.ecb.europa.eu/press/key/date/1997/...,S,html
4,5,1997-05-13,Alexandre Lamfalussy,The European Central Bank: independent and acc...,"{""central"":33,""monetary"":30,""policy"":29,""escb""...",EN,https://www.ecb.europa.eu/press/key/date/1997/...,S,html


In [9]:
from gather_data import DataDownloader
import numpy as np
from pathlib import Path
datadownloader = DataDownloader()

for index, row in datadownloader.metadata_df[datadownloader.metadata_df['file_ending'].isna()].iterrows():
    speech_id = row['speech_id']
    url = row['what_weblink']
    file_ending = row['file_ending']
    if file_ending is np.nan:
        file_ending = 'pdf'
    date = row['when_speech'].strftime('%Y-%m-%d')
    type_long = row['what_type_long']
    filepath = datadownloader.output_dir / f"{speech_id}_{date}_{type_long}.{file_ending}"
    datadownloader._download_file(url, filepath)

2025-02-16 12:12:46,647 - INFO - Downloaded https://www.ecb.europa.eu/press/key/date/2018/html/ecb.sp180615_slides.en.pdf?78c96a75d5ccb30bb7105d3fb28d7612 to C:\Users\povhi\OneDrive\01_Uni\03_TU Wien\MSc Data Science\09_Interdisciplinary_Project\ECB_Topic_Chronicles\data\speeches\2484_2018-06-15_speech.pdf
2025-02-16 12:12:46,823 - INFO - Downloaded https://www.ecb.europa.eu/press/key/date/2018/html/ecb.sp180828.en.pdf?aeeb0c5dc9dfdbed5c682ed2ab624f76 to C:\Users\povhi\OneDrive\01_Uni\03_TU Wien\MSc Data Science\09_Interdisciplinary_Project\ECB_Topic_Chronicles\data\speeches\2500_2018-08-28_speech.pdf
2025-02-16 12:12:47,137 - INFO - Downloaded https://www.ecb.europa.eu/press/key/date/2018/html/ecb.sp180917_2.en.pdf?4369df2ed332e17efde72713004966a1 to C:\Users\povhi\OneDrive\01_Uni\03_TU Wien\MSc Data Science\09_Interdisciplinary_Project\ECB_Topic_Chronicles\data\speeches\2508_2018-09-17_speech.pdf
2025-02-16 12:12:47,353 - INFO - Downloaded https://www.ecb.europa.eu/press/key/date/201

In [ ]:
import bs4
from bs4 import BeautifulSoup

with open(r"data\speeches\1_1997-02-07_speech.html", "r", encoding="utf-8") as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')



['ecb-pressContentTitle']

In [66]:
from pathlib import Path
from gather_data import Metadata
metadata = Metadata()
directory = "data/speeches"
h1_tags = {x: [] for x in list(metadata.what_type_dict.values())}
for html_file in Path(directory).rglob("*.html"):
    speech_type = str(html_file).split("_")[-1].split(".")[0]
    with open(html_file, "r", encoding="utf-8") as f:
        html_content = f.read()
    temp_soup = BeautifulSoup(html_content, 'html.parser')
    h1_tags[speech_type].extend(temp_soup.find('h1').get('class', ['no_class']))
h1_tags = {k: sorted(list(set(v))) for k,v in h1_tags.items()}

KeyboardInterrupt: 

In [3]:
from pathlib import Path
from gather_data import Metadata
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

metadata = Metadata()
directory = "data/speeches"
h1_tags = {x: [] for x in list(metadata.what_type_dict.values())}

def process_html_file(html_file):
    speech_type = str(html_file).split("_")[-1].split(".")[0]
    with open(html_file, "r", encoding="utf-8") as f:
        html_content = f.read()
    temp_soup = BeautifulSoup(html_content, 'html.parser')
    return speech_type, temp_soup.find('h1').get('class', ['no_class'])

html_files = list(Path(directory).rglob("*.html"))

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = {executor.submit(process_html_file, html_file): html_file for html_file in html_files}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing HTML files"):
        speech_type, classes = future.result()
        h1_tags[speech_type].extend(classes)

h1_tags = {k: sorted(list(set(v))) for k, v in h1_tags.items()}

Processing HTML files: 100%|██████████| 3869/3869 [06:32<00:00,  9.87it/s] 


In [5]:
h1_tags

{'speech': ['ecb-pressContentTitle', 'ecb0', 'no_class'],
 'interview': ['ecb-pressContentTitle', 'no_class'],
 'press-conference': ['ecb-pressContentTitle', 'no_class'],
 'blog-post': ['ecb-pressContentTitle', 'no_class'],
 'podcast': ['no_class']}

In [15]:
from bs4 import BeautifulSoup
from pathlib import Path
import re
def extract_main_content(html_file, classes_to_remove, texts_to_remove, div_classes_to_remove):
    with open(html_file, "r", encoding="utf-8") as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    result = {"content": "", "related_topics": []}
   
    # Extract related topics, has to be done before the rest otherwise its eaten from the soup already
    related_topics_div = soup.find('div', class_='related-topics')
    if related_topics_div:
        result["related_topics"] = [li.get_text(strip=True) for li in related_topics_div.find_all('li')]
    if main_content:
        # Remove paragraphs with the specified classes
        for class_to_remove in classes_to_remove:
            for tag in main_content.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'], class_=class_to_remove):
                tag.decompose()
       
        # Remove divs with the specified classes
        for div_class_to_remove in div_classes_to_remove:
            for div in main_content.find_all('div', class_=div_class_to_remove):
                div.decompose()
                
        # Remove paragraphs containing specific text and stop processing further paragraphs
        stop_processing = False
        for p in main_content.find_all('p'):
            if stop_processing:
                p.decompose()
            elif any(text_to_remove in p.get_text() for text_to_remove in texts_to_remove):
                p.decompose()
                stop_processing = True
       
        # Extract text from the main content
        content = main_content.get_text(separator='\n', strip=True)
        # Remove multiple whitespaces and linebreaks 
        content = re.sub(r'\s+', ' ', content)
        result["content"] = content

    return result

# Example usage
# html_file = Path(r'data\speeches\4770_2025-01-09_interview.html')
# html_file = Path(r'data\speeches\4782_2025-01-27_speech.html')
html_file = Path(r'data\speeches\4790_2025-02-05_blog-post.html')
# html_file = Path(r'data\speeches\4644_2024-06-06_press-conference.html')

classes_to_remove = ["ecb-publicationDate", "ecb-authors", "ecb-pressContentSubtitle"]
texts_to_remove = [
    "The views expressed in each blog entry are those of the author(s) and do not necessarily represent the views of the European Central Bank and the Eurosystem.",
]
div_classes_to_remove = ["related-topics", "title", "address-box", "footnotes", "final hidden"]
result = extract_main_content(html_file, classes_to_remove, texts_to_remove, div_classes_to_remove)
print(result['content'])
print(result['related_topics'])

The monetary policies of the ECB and the US Federal Reserve are not always in sync. But how does the Fed’s policy affect the euro area economy? This ECB Blog looks at how monetary policy in the United States travels across the Atlantic and what this means for the ECB. The global economy is interconnected. Central banks’ monetary policies therefore often affect other economies. These “monetary policy spillovers” are particularly relevant for economies as closely linked as the United States and the euro area. Spillovers from US monetary policy initially work in the opposite direction to ECB monetary policy, but then later in the same direction. For instance, a surprise tightening of US policy leads to an initial increase in euro area inflation as the euro weakens. However, over time, tighter US monetary policy drags down euro area inflation much like tighter ECB policy would. In more detail, when the Fed unexpectedly raises interest rates to curb inflation and economic activity, euro are

In [ ]:
import re
class Parser: 
    def __init__(self, dir = Path('data/speeches')):
        self.dir = dir
        self.classes_to_remove = ["ecb-publicationDate", "ecb-authors", "ecb-pressContentSubtitle"]
        # If the following text is found the parser stops processing the remaining paragraphs
        self.texts_to_remove = [
            "The views expressed in each blog entry are those of the author(s) and do not necessarily represent the views of the European Central Bank and the Eurosystem.",
        ]
        self.div_classes_to_remove = ["related-topics", "title", "address-box", "footnotes", "final hidden"]

    def extract_main_content_from_html(self, html_file_path):
        with open(html_file_path, "r", encoding="utf-8") as f:
            html_content = f.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        main_content = soup.find('main')
        result = {"content": "", "related_topics": []}
    
        # Extract related topics, has to be done before the rest otherwise its eaten from the soup already
        related_topics_div = soup.find('div', class_='related-topics')
        if related_topics_div:
            result["related_topics"] = [li.get_text(strip=True) for li in related_topics_div.find_all('li')]
        if main_content:
            # Remove paragraphs with the specified classes
            for class_to_remove in self.classes_to_remove:
                for tag in main_content.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'], class_=class_to_remove):
                    tag.decompose()
        
            # Remove divs with the specified classes
            for div_class_to_remove in self.div_classes_to_remove:
                for div in main_content.find_all('div', class_=div_class_to_remove):
                    div.decompose()
                    
            # Remove paragraphs containing specific text and stop processing further paragraphs
            stop_processing = False
            for p in main_content.find_all('p'):
                if stop_processing:
                    p.decompose()
                elif any(text_to_remove in p.get_text() for text_to_remove in self.texts_to_remove):
                    p.decompose()
                    stop_processing = True
        
            # Extract text from the main content
            content = main_content.get_text(separator='\n', strip=True)
            # Remove multiple whitespaces and linebreaks 
            content = re.sub(r'\s+', ' ', content)
            result["content"] = content
    
        return result
    
    def process_directory(self):
        
        for html_file in Path(self.dir).rglob("*.html"):
            result = self.extract_main_content_from_html(html_file)

In [3]:
def extract_main_content(html_file, remove_qa=False):
    """
    Extract main content and related topics from HTML files.
    
    Args:
        html_file: Path to HTML file
        remove_qa: Boolean to indicate if Q&A sections should be removed
    
    Returns:
        dict with 'content' and 'related_topics' keys
    """
    with open(html_file, "r", encoding="utf-8") as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    
    result = {"content": "", "related_topics": []}
    
    # Find main content
    main_content = soup.find('main')
    if not main_content:
        return result
        
    # Remove unnecessary elements
    elements_to_remove = [
        'title',
        'ecb-pressContentTitle',
        'ecb-pressContentSubtitle', 
        'ecb-pressContentPubDate',
        'ecb-publicationDate',
        'author',
        'author-image',
        'author-details',
        'address-box',
        'footnotes',
        'see-also-boxes', 
        'side-authors'
    ]
    
    for class_name in elements_to_remove:
        for element in main_content.find_all(class_=class_name):
            element.decompose()
            
    # Remove header elements that typically contain titles/dates
    for header in main_content.find_all(['h1', 'h2', 'h3']):
        if any(text in header.get_text().lower() for text in ['introduction', 'conclusion']):
            continue
        header.decompose()
            
    # Extract related topics
    related_topics_div = main_content.find('div', class_='related-topics')
    if related_topics_div:
        result["related_topics"] = [
            li.get_text(strip=True) 
            for li in related_topics_div.find_all('li')
        ]
        related_topics_div.decompose()
    
    # Clean up the remaining content
    content_text = []
    for p in main_content.find_all(['p']):
        # Skip author/date information
        if any(marker in p.get_text().lower() for marker in ['frankfurt', 'blog post by']):
            continue
            
        # Add paragraph text
        text = p.get_text(strip=True)
        if text:
            content_text.append(text)
            
    # Join all content
    result["content"] = '\n\n'.join(content_text)
    
    return result

# Example usage
html_file = Path(r'data\speeches\4770_2025-01-09_interview.html')
html_file = Path(r'data\speeches\4782_2025-01-27_speech.html')
html_file = Path(r'data\speeches\4790_2025-02-05_blog-post.html')
# html_file = Path(r'data\speeches\4644_2024-06-06_press-conference.html')

classes_to_remove = ["ecb-publicationDate", "ecb-authors", "ecb-pressContentSubtitle"]
texts_to_remove = [
    "The views expressed in each blog entry are those of the author(s) and do not necessarily represent the views of the European Central Bank and the Eurosystem.",
]
div_classes_to_remove = ["related-topics", "title", "address-box", "footnotes", "final hidden"]
result = extract_main_content(html_file)
print(result['content'])
print(result['related_topics'])

By Stefan Gebauer, Georgios Georgiadis, Fédéric Holm-Hadulla and Thomas Kostka

The monetary policies of the ECB and the US Federal Reserve are not always in sync. But how does the Fed’s policy affect the euro area economy? This ECB Blog looks at how monetary policy in the United States travels across the Atlantic and what this means for the ECB.

The global economy is interconnected.Central banks’ monetary policiestherefore often affect other economies. These “monetary policy spillovers” are particularly relevant for economies as closely linked as the United States and the euro area. Spillovers from US monetary policy initially work in the opposite direction to ECB monetary policy, but then later in the same direction. For instance, a surprise tightening of US policy leads to an initial increase in euro area inflation as the euro weakens. However, over time, tighter US monetary policy drags down euro area inflation much like tighter ECB policy would.

In more detail, when the Fed unex